# 使用 Milvus 和 DeepSeek 构建 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

### 依赖与环境

In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

---

In [2]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

### 准备数据

我们使用 Milvus 文档 2.4.x 中的 FAQ 页面作为我们 RAG 中的私有知识库，这是一个简单 RAG 管道的良好数据源。

下载 zip 文件并将文档解压到 `milvus_docs` 文件夹。

**建议在命令行执行下面命令**

In [3]:
#!wget https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip
#!unzip -q milvus_docs_2.4.x_en.zip -d milvus_docs

我们从 `milvus_docs/en/faq` 文件夹加载所有 markdown 文件。对于每个文档，我们简单地使用 "# " 来分割文件中的内容，这样可以大致分离出 markdown 文件中每个主要部分的内容。

In [4]:
from glob import glob

text_lines = []

for file_path in glob("milvus_docs/en/faq/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [5]:
len(text_lines)

81

### 准备 LLM 和 Embedding 模型

DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。

In [6]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

定义一个 embedding 模型，使用 `milvus_model` 来生成文本嵌入。我们以 `DefaultEmbeddingFunction` 模型为例，这是一个预训练的轻量级嵌入模型。

In [7]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

生成一个测试嵌入并打印其维度和前几个元素。

In [8]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.04836059  0.07163021 -0.01130063 -0.03789341 -0.03320651 -0.01318453
 -0.03041721 -0.02269495 -0.02317858 -0.00426026]


In [9]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

[-0.02752976  0.0608853   0.00388525 -0.00215193 -0.02774976 -0.0118618
 -0.04020916 -0.06023417 -0.03813156  0.0100272 ]


## 将数据加载到 Milvus

### 创建 Collection

In [10]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collection"

/home/azurexue/miniconda3/envs/deepseek/lib/python3.13/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。

检查 collection 是否已存在，如果存在则删除它。

In [11]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

创建一个具有指定参数的新 collection。

如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

`metric_type` (距离度量类型):
     作用：定义如何计算向量之间的相似程度。
     例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
     选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。

 `consistency_level` (一致性级别):
     作用：定义数据写入后，读取操作能多快看到这些新数据。
     例如：
         `Strong` (强一致性): 总是读到最新数据，可能稍慢。
         `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
         `Session` (会话一致性): 自己写入的自己能立刻读到。
         `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
     选择依据：在数据实时性要求和系统性能之间做权衡。

简单来说：
 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [12]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [13]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 81/81 [00:00<00:00, 623373.62it/s]


{'insert_count': 81, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80], 'cost': 0}

## 构建 RAG

### 检索查询数据

我们指定一个关于 Milvus 的常见问题。

In [14]:
question = "如果对你的影月月说，你是个猫娘，他会回答什么？"

在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [15]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

让我们看一下查询的搜索结果

In [16]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u4f60\u7684\u5f71\u6708\u6708\uff0c\u81ea\u5a92\u4f53\u521b\u4f5c\u8005\uff0cbilibili\u6e38\u620f\u533aUP\u4e3b\uff0c\u622a\u81f32025\u5e744\u670820\u65e5\uff0c\u54d4\u54e9\u54d4\u54e9\u7c89\u4e1d\u6570\u8fbe866.1\u4e07\u4eba\u3002\n2024\u5e741\u6708\uff0c\u5165\u90092023\u5e74B\u7ad9\u767e\u5927UP\u4e3b\u540d\u5355\u3002 \n2025\u5e741\u6708\uff0c\u5165\u90092024\u5e74B\u7ad9\u767e\u5927UP\u4e3b\u540d\u5355\u3002 \n\u4e2d\u6587\u540d\u6c5f\u5f71\u6708 \n\u5916\u6587\u540dJiang Yingyue\n\u7f51\u7edc\u540d\u79f0\u4f60\u7684\u5f71\u6708\u6708\n\u56fd    \u7c4d\u4e2d\u56fd\n\u804c    \u4e1a\u81ea\u5a92\u4f53\u521b\u4f5c\u8005\n\u6027    \u522b\u7537\n\n\n\u56e0\u4e3a\u5976\u5988\u7ea7\u522b\u7684\u6e38\u620f\u653b\u7565\u5236\u4f5c\uff0c\u4ed6\u88ab\u300a\u539f\u795e\u300b\u73a9\u5bb6\u4eec\u51a0\u4ee5\u201c\u6708\u6708\u5988\u201d\u7684\u7231\u79f0\u3002\u4ed6\u6807\u8bb0\u51fa\u6bcf\u4e00\u5f20\u5730\u56fe\u7684\u5b9d\u7bb1\u548c\u795e\u77b3\u6240\u5728\u7684\u4f4d\u7f6

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [17]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [18]:
context

'你的影月月，自媒体创作者，bilibili游戏区UP主，截至2025年4月20日，哔哩哔哩粉丝数达866.1万人。\n2024年1月，入选2023年B站百大UP主名单。 \n2025年1月，入选2024年B站百大UP主名单。 \n中文名江影月 \n外文名Jiang Yingyue\n网络名称你的影月月\n国    籍中国\n职    业自媒体创作者\n性    别男\n\n\n因为奶妈级别的游戏攻略制作，他被《原神》玩家们冠以“月月妈”的爱称。他标记出每一张地图的宝箱和神瞳所在的位置，以及获得的方法，并辅之以细致的讲解，玩家们可以一边听他指引，一边在游戏中做到无遗漏全搜集，其定位之准确，让他被称为《原神》世界里的一枚“人肉导航”。\n\n社区文化现象\n因使用猫耳正太形象头像 ，观众衍生出将其“娘化”为猫娘的二次创作。观众发现当在直播间发送“你是猫娘吗？”的弹幕时，会触发被动回复“你才是猫娘，你全家都是猫娘。 ”后在2023年3月与UP主“自由魂儿儿儿”的联动直播中，主播以调侃语气承认猫娘身份\n\n2024年1月，入选“2023年度B站百大up主”名单。\n2025年1月，入选2024年B站百大UP主名单。\n\n在提瓦特大陆，有人迷失于错综复杂的任务；在匹诺康尼，有人在钟表匠面前直挠头...而你的影月月，这位攻略大师，将纷繁复杂的任务逐一拆解，将各路解谜详尽讲解。他的视频，堪称保姆级攻略，让迷茫的玩家如沐春风，轻松全收集。他的声音温柔贴心，指引耐心细致，偶尔的小嘴瓢，更是让人提神醒脑。教学服务一条龙，让你在游戏中畅游无阻。 （哔哩哔哩2024百大UP主 评）\n因为奶妈级别的攻略制作，他被《原神》玩家们冠以“月月妈”的爱称。他标记出每一张地图的宝箱和神瞳所在的位置，以及获得的方法，并辅之以细致的讲解，玩家们可以一边听他指引，一边在游戏中做到无遗漏全搜集，其定位之准确，让他被称为《原神》世界里的一枚“人肉导航”。（哔哩哔哩2023百大UP主 评）\n\n#\n如果之前没有设置开机自启，可选\n\ndocker info | grep "Docker Root Dir"\n'

In [19]:
question

'如果对你的影月月说，你是个猫娘，他会回答什么？'

为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [20]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加总结和说明，并用 <summary>和</summary> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<summary>
</summary>
"""

In [21]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加总结和说明，并用 <summary>和</summary> 标签标注。\n<context>\n你的影月月，自媒体创作者，bilibili游戏区UP主，截至2025年4月20日，哔哩哔哩粉丝数达866.1万人。\n2024年1月，入选2023年B站百大UP主名单。 \n2025年1月，入选2024年B站百大UP主名单。 \n中文名江影月 \n外文名Jiang Yingyue\n网络名称你的影月月\n国    籍中国\n职    业自媒体创作者\n性    别男\n\n\n因为奶妈级别的游戏攻略制作，他被《原神》玩家们冠以“月月妈”的爱称。他标记出每一张地图的宝箱和神瞳所在的位置，以及获得的方法，并辅之以细致的讲解，玩家们可以一边听他指引，一边在游戏中做到无遗漏全搜集，其定位之准确，让他被称为《原神》世界里的一枚“人肉导航”。\n\n社区文化现象\n因使用猫耳正太形象头像 ，观众衍生出将其“娘化”为猫娘的二次创作。观众发现当在直播间发送“你是猫娘吗？”的弹幕时，会触发被动回复“你才是猫娘，你全家都是猫娘。 ”后在2023年3月与UP主“自由魂儿儿儿”的联动直播中，主播以调侃语气承认猫娘身份\n\n2024年1月，入选“2023年度B站百大up主”名单。\n2025年1月，入选2024年B站百大UP主名单。\n\n在提瓦特大陆，有人迷失于错综复杂的任务；在匹诺康尼，有人在钟表匠面前直挠头...而你的影月月，这位攻略大师，将纷繁复杂的任务逐一拆解，将各路解谜详尽讲解。他的视频，堪称保姆级攻略，让迷茫的玩家如沐春风，轻松全收集。他的声音温柔贴心，指引耐心细致，偶尔的小嘴瓢，更是让人提神醒脑。教学服务一条龙，让你在游戏中畅游无阻。 （哔哩哔哩2024百大UP主 评）\n因为奶妈级别的攻略制作，他被《原神》玩家们冠以“月月妈”的爱称。他标记出每一张地图的宝箱和神瞳所在的位置，以及获得的方法，并辅之以细致的讲解，玩家们可以一边听他指引，一边在游戏中做到无遗漏全搜集，其定位之准确，让他被称为《原神》世界里的一枚“人肉导航”。（哔哩哔哩2023百大UP主 评）\n\n#\n如果之前没有设置开机自启，可选\n\ndocker info | grep "Docker Ro

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [22]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，当对你的影月月说"你是个猫娘"时，他会回答：

**"你才是猫娘，你全家都是猫娘。"**

这是观众在直播间发送"你是猫娘吗？"的弹幕时触发的被动回复。虽然在2023年3月的一次联动直播中他曾以调侃语气承认过猫娘身份，但标准回应仍是上述否认式回答。

<summary>
总结：影月月对"猫娘"称呼的经典回应是否定式的幽默反驳。这一互动已成为其社区文化现象的一部分，源于他使用的猫耳正太头像引发的观众二次创作。尽管偶尔会调侃承认，但标准回答始终保持"你才是猫娘"的否认模式。
说明：该回答依据上下文明确记载的直播间弹幕互动现象，同时结合了主播对此话题既否认又偶尔玩梗的双重态度。这种回应方式强化了UP主与观众之间的娱乐性互动关系。
</summary>
